In [2]:
# API Key를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API Key 정보로드
load_dotenv()

import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_study"

In [3]:
from langchain_community.llms import Ollama

llm_aya = Ollama(model="aya:8b-23-q8_0", temperature=0)
llm_llama3 = Ollama(model="llama3:8b-instruct-q8_0", temperature=0)
llm_gemma = Ollama(model="gemma:7b-instruct-q8_0", temperature=0)


In [4]:
from langchain_openai import ChatOpenAI

llm_gpt = ChatOpenAI(model="gpt-4o", temperature=0)

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    System : 너는 다음 Instruction을 잘 수행하는 assistant 이다.
    Instruction : {instruction}
    """
)

In [7]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [31]:
chain_aya = prompt | llm_aya | output_parser
chain_llama3 = prompt | llm_llama3 | output_parser
chain_gemma = prompt | llm_gemma | output_parser
chain_gpt = prompt | llm_gpt | output_parser

param = {
    "aya:8b-23-q8_0" : chain_aya ,
    "llama3:8b-instruct-q8_0" : chain_llama3 ,
    "gemma:7b-instruct-q8_0" : chain_gemma ,
    "gpt-4o" : chain_gpt ,
}

from langchain_core.runnables import RunnableParallel

chain_llms = RunnableParallel(**param)

In [32]:
# chain_llms 실행 Test
responses = chain_llms.invoke({"instruction":"HOT6 라는 음료수에 대해 알려주세요."})

from pprint import pprint
pprint(responses)

{'aya:8b-23-q8_0': 'HOT6은 시중에서 판매되는 인기 있는 음료수 브랜드입니다. 이 음료수는 주로 탄산수와 과일 향의 '
                   '혼합물로 구성되어 있으며, 신맛과 단맛의 균형이 잘 잡혀 있어 많은 사람들에게 사랑받고 있습니다.\n'
                   '\n'
                   'HOT6은 다양한 맛으로 출시되어 있으며, 가장 인기 있는 제품으로는 레몬라임 맛, 오렌지 맛, 자몽 맛 '
                   '등이 있습니다. 이 음료수는 일반적으로 병이나 캔으로 판매되며, 탄산수가 함유되어 있어 상쾌한 맛과 '
                   '청량감을 제공합니다.\n'
                   '\n'
                   'HOT6은 저칼로리와 저당도로 알려져 있으며, 건강을 고려하는 소비자들에게 인기 있는 선택肢가 되고 '
                   '있습니다. 또한, 카페인이 함유되어 있지 않아 밤에도 편안하게 즐길 수 있는 음료수이기도 합니다.\n'
                   '\n'
                   '이 브랜드는 다양한 마케팅 캠페인과 광고 활동을 통해 젊은 층과 건강에 관심이 많은 소비자들 사이에서 '
                   '인기를 얻고 있습니다. HOT6은 일반적으로 슈퍼마켓, 편의점, 레스토랑 등 다양한 소매점에서 쉽게 '
                   '찾아볼 수 있으며, 때로는 프로모션이나 할인 행사도 진행되기도 합니다.\n'
                   '\n'
                   'HOT6을 선택할 때는 개인적인 기호에 따라 맛과 향을 고려하는 것이 좋습니다. 또한, 탄산 음료를 즐길 '
                   '때에는 물을 충분히 마셔 두어 신체에 수분을 공급하고 탄산 가스의 부작용을 줄이는 것이 좋습니다.',
 'gemma:7b-instruct-q

In [10]:
eval_prompt = PromptTemplate(
    template="""
    System : 
    너는 llm model들의 답변을 비교하고 평가하는 AI 이다.
    Instruction과 Responses 안의 각각의 llm별 응답을 
    정확성(Accuracy), 관련성(Relevance), 유창성(Fluency), 완전성(Completeness) 측면에서    
    분석하고 최고 점수 5점으로 0점 ~ 5점 사이 점수를 부여하라.

    한국어로 답변해줘.

    Instruction : {instruction}
    Resonses : {responses}
    """,
    input_variables=["instruction", "responses"],
    #partial_variables={"instruction" : instruction},
)

In [11]:
chain_eval = eval_prompt | llm_gpt

In [12]:
from langchain_core.runnables import RunnablePassthrough

chain_combinded = (
    {"responses" : chain_llms, "instruction" : RunnablePassthrough()}
    | chain_eval
)

In [13]:
instruction = "1990년대 한국 대중가요에 대해 알려주세요."

response = chain_combinded.invoke({"instruction":instruction})

In [14]:
print(response.content)

### 평가 기준
1. **정확성(Accuracy)**: 제공된 정보가 사실에 근거하고 정확한지 평가합니다.
2. **관련성(Relevance)**: 응답이 주어진 질문에 얼마나 잘 부합하는지 평가합니다.
3. **유창성(Fluency)**: 응답이 문법적으로 올바르고 읽기 쉬운지 평가합니다.
4. **완전성(Completeness)**: 응답이 질문에 대해 충분히 상세하고 포괄적인지 평가합니다.

### 평가

#### aya
- **정확성(Accuracy)**: 5점
  - 1990년대 한국 대중가요의 주요 특징과 아티스트들을 정확하게 설명하고 있습니다.
- **관련성(Relevance)**: 5점
  - 질문에 대한 답변으로 매우 적절하며, 1990년대 한국 대중가요의 다양한 측면을 다루고 있습니다.
- **유창성(Fluency)**: 5점
  - 문법적으로 올바르고 읽기 쉬운 문장으로 구성되어 있습니다.
- **완전성(Completeness)**: 5점
  - 발라드, 댄스 음악, 그룹과 여성 가수, 음악 장르의 다양성, 음악 프로그램과 콘서트, 사회적 이슈 등 다양한 측면을 포괄적으로 다루고 있습니다.

**총점: 20점**

#### gemma
- **정확성(Accuracy)**: 3점
  - 일부 정보는 정확하지만, 연극의 발달과 같은 내용은 다소 부정확하거나 관련성이 떨어집니다.
- **관련성(Relevance)**: 3점
  - 질문에 대한 답변으로 적절하지만, 연극과 같은 비관련 정보가 포함되어 있습니다.
- **유창성(Fluency)**: 4점
  - 문법적으로 올바르지만, 일부 문장은 다소 어색하게 느껴질 수 있습니다.
- **완전성(Completeness)**: 3점
  - 주요 특징과 사건을 다루고 있지만, 구체적인 아티스트나 곡에 대한 언급이 부족합니다.

**총점: 13점**

#### llama3
- **정확성(Accuracy)**: 4점
  - 대부분의 정보가 정확하지만, TVXQ의 데뷔 연도(2003년)는 1990년대와 관련이